# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV

Открываем файл

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

Изучаем файл

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

### Вывод

Мы видим таблицу (3214 х 5). Данные предобработаны. Мы только изменили тип данных в столбцах calls и messages на int64. Каждый объект в наборе данных - это информация о поведении одного пользователя за месяц.<br><br>
Известно, что:
 - сalls — количество звонков,
 - minutes — суммарная длительность звонков в минутах,
 - messages — количество sms-сообщений,
 - mb_used — израсходованный интернет-трафик в Мб,
 - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Соотношение пользователей тарифов: «Ультра» — 30%, «Смарт» — 70%. Это говорит о том, что у нас в целевом признаке присутствует дисбаланс, то есть наша модель будет предвзята в сторону тарифа «Смарт». Предполагаю, что дисбаланс недостаточно ярковыраженн, чтобы была необходимость с ним бороться.

## Разбиваем данные на выборки

Разбиваем датафрейм на 3 выборки: обучающую - 60% (df_trn), валидационную - 20% (df_vld) и тестовую - 20% (df_tst)

In [7]:
df_trn, df_vld, df_tst = np.split(df.sample(frac=1, random_state=12345), [int(.6*len(df)), int(.8*len(df))])

Проверяем результат

In [8]:
df_trn.shape

(1928, 5)

In [9]:
df_vld.shape

(643, 5)

In [10]:
df_tst.shape

(643, 5)

## Исследуем модели

## Исследуйте модели

### Формируем признаки и целевой признак

Формируем признаки и целевой признак на базе обучающей выборки (df_trn)

In [11]:
features_trn = df_trn.drop(['is_ultra'], axis=1)
target_trn = df_trn['is_ultra']

Формируем признаки и целевой признак на базе валидационной выборки (df_vld)

In [12]:
features_vld = df_vld.drop(['is_ultra'], axis=1)
target_vld = df_vld['is_ultra']

Формируем признаки и целевой признак на базе тестовой выборки (df_tst)

In [13]:
features_tst = df_tst.drop(['is_ultra'], axis=1)
target_tst = df_tst['is_ultra']

### Модель Логическая регрессия

In [14]:
# Инициализируем модель
model_LR = LogisticRegression()

# Обучаем модель
model_LR.fit(features_trn, target_trn)

# Cчитаем предсказания и точность полученной классификации
predict_LR_tst = model_LR.predict(features_tst)
predict_LR_vld = model_LR.predict(features_vld)

Создадим датафрейм model, в который будем добавлять результаты каждой модели, чтобы потом выбрать лучшие

In [15]:
model_data = {
    'name':['Логическая Регрессия'],
    'model_var':np.nan,
    'test_accuracy':[accuracy_score(predict_LR_tst,target_tst)],
    'validate_accuracy':[accuracy_score(predict_LR_vld,target_vld)]}
model = pd.DataFrame(model_data)
model

,name,model_var,test_accuracy,validate_accuracy
0,Логическая Регрессия,NaN,0.738725,0.713841


Точность модели Логическая регрессия - 0.714

In [16]:
top_models = model

### Модель Дерево решений

In [17]:
for depth in range(1,100,1):
    
    # Инициализируем модель
    model_DT=DecisionTreeClassifier(random_state=12345, max_depth=depth)
    
    # Обучаем модель
    model_DT.fit(features_trn,target_trn)
    
    # Cчитаем предсказания и точность полученной классификации
    prediction_DT_tst=model_DT.predict(features_tst)
    prediction_DT_vld=model_DT.predict(features_vld)
    
    # Добавляем результаты в датафрейм model
    name = 'Дерево Решений'
    model_var = 'm_depth: {}'.format(depth)
    model_data = {
        'name':name,
        'model_var':model_var,
        'test_accuracy':accuracy_score(prediction_DT_tst,target_tst),
        'validate_accuracy':accuracy_score(prediction_DT_vld,target_vld)}
    model = model.append(model_data, ignore_index=True)

C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_index=True)
C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_index=True)
C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_index=True)
C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_

C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_index=True)
C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_index=True)
C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_index=True)
C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_

C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_index=True)
C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_index=True)
C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_index=True)
C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\60141266.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(model_data, ignore_

Сортируем датафрейм model и выводим 10 лучших результатов

In [18]:
model.sort_values('validate_accuracy',ascending=False).head(10)

,name,model_var,test_accuracy,validate_accuracy
10,Дерево Решений,m_depth: 10,0.794712,0.777605
9,Дерево Решений,m_depth: 9,0.794712,0.776050
4,Дерево Решений,m_depth: 4,0.790047,0.774495
5,Дерево Решений,m_depth: 5,0.788491,0.772939
6,Дерево Решений,m_depth: 6,0.793157,0.769829
11,Дерево Решений,m_depth: 11,0.783826,0.769829
12,Дерево Решений,m_depth: 12,0.776050,0.769829
3,Дерево Решений,m_depth: 3,0.794712,0.766719
8,Дерево Решений,m_depth: 8,0.783826,0.765163
13,Дерево Решений,m_depth: 13,0.774495,0.763608


Лучшая точность (0.778) достигнута у модели Дерево решений, при значении максимальной глубины дерева 10.

In [19]:
temp = [top_models, model.sort_values('validate_accuracy',ascending=False).head(1)]
top_models = pd.concat(temp)
top_models

,name,model_var,test_accuracy,validate_accuracy
0,Логическая Регрессия,NaN,0.738725,0.713841
10,Дерево Решений,m_depth: 10,0.794712,0.777605


### Модель Случайный лес

In [20]:
%%time

for est in range(1,50,1):
    for depth in range(1,11):    
        
         # Инициализируем модель
        model_RF=RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345, min_samples_leaf=2)
        
        # Обучаем модель
        model_RF.fit(features_trn,target_trn)
        
        # Cчитаем предсказания и точность полученной классификации
        prediction_RF_tst=model_RF.predict(features_tst)
        prediction_RF_vld=model_RF.predict(features_vld)
        
        # Добавляем результаты в датафрейм model
        name = 'Случайный Лес'
        model_var = 'n_est: {}'.format(est),' m_depth: {}'.format(depth)
        model_data = {
            'name':name,
            'model_var':model_var,
            'test_accuracy':accuracy_score(prediction_RF_tst,target_tst),
            'validate_accuracy':accuracy_score(prediction_RF_vld,target_vld)}
        model = model.append(model_data, ignore_index=True)

<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from panda

<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from panda

<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from panda

<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from panda

<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from panda

...

<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from panda

<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from panda

<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from panda

CPU times: total: 47.2 s
Wall time: 48.1 s


<timed exec>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [21]:
model.sort_values(['validate_accuracy'], ascending=False).head(10)

,name,model_var,test_accuracy,validate_accuracy
309,Случайный Лес,"(n_est: 21, m_depth: 10)",0.804044,0.807154
349,Случайный Лес,"(n_est: 25, m_depth: 10)",0.808709,0.807154
329,Случайный Лес,"(n_est: 23, m_depth: 10)",0.804044,0.804044
379,Случайный Лес,"(n_est: 28, m_depth: 10)",0.808709,0.804044
279,Случайный Лес,"(n_est: 18, m_depth: 10)",0.800933,0.804044
359,Случайный Лес,"(n_est: 26, m_depth: 10)",0.808709,0.802488
289,Случайный Лес,"(n_est: 19, m_depth: 10)",0.802488,0.802488
299,Случайный Лес,"(n_est: 20, m_depth: 10)",0.800933,0.802488
319,Случайный Лес,"(n_est: 22, m_depth: 10)",0.799378,0.802488
339,Случайный Лес,"(n_est: 24, m_depth: 10)",0.804044,0.802488


Лучшая точность (0.807) достигнута у модели Случайный лес, при значениях:
 - максимальная глубина деревьев - 10.
 - число деревьев - 25.
 - ограничении на число объектов в листьях - 2

In [22]:
temp = [top_models, model.sort_values('validate_accuracy',ascending=False).head(1)]
top_models = pd.concat(temp)
top_models

,name,model_var,test_accuracy,validate_accuracy
0,Логическая Регрессия,NaN,0.738725,0.713841
10,Дерево Решений,m_depth: 10,0.794712,0.777605
309,Случайный Лес,"(n_est: 21, m_depth: 10)",0.804044,0.807154


In [23]:
model.shape

(590, 4)

In [24]:
top_models

,name,model_var,test_accuracy,validate_accuracy
0,Логическая Регрессия,NaN,0.738725,0.713841
10,Дерево Решений,m_depth: 10,0.794712,0.777605
309,Случайный Лес,"(n_est: 21, m_depth: 10)",0.804044,0.807154


### Вывод

После исследования 3-х моделей, мы полечили 590 интераций. Лучшей оказалась Случайный лес с точностью 0.807 при значениях: максимальной глубины деревьев 10, числа деревьев 25 и ограничения на число объектов в листьях 2.

## Проверим модель на тестовой выборке

Для сдачи проекта, доля правильных ответов должна быть по крайней мере 75%.<br>
Ранее мы уже проверили наши модели на тестовых выборках и получили следующие результаты у лучших моделей:

In [25]:
top_models

,name,model_var,test_accuracy,validate_accuracy
0,Логическая Регрессия,NaN,0.738725,0.713841
10,Дерево Решений,m_depth: 10,0.794712,0.777605
309,Случайный Лес,"(n_est: 21, m_depth: 10)",0.804044,0.807154


### Вывод

 Модели Дерево решений и Случайный лес соответствуют условию точности (доля правильных ответов более 75%)

## Проверяем модель на адекватность

Для проверки модели на адекватность, создадим baseline-модель. Пусть это будет Dummy классификатор. Если наша более сложная модель действительно дает нам прирост качества, то её точность должна быть выше.

In [26]:
# Инициализируем модель
model_DC = DummyClassifier(strategy='most_frequent', random_state=12345)

# Обучаем модель
model_DC.fit(features_trn, target_trn)

# Cчитаем предсказания и точность полученной классификации
predict_DC_tst = model_DC.predict(features_tst)
predict_DC_vld = model_DC.predict(features_vld)

# Добавляем результаты в датафрейм top_models
name = 'Dummy классификатор'
model_var = np.nan
model_data = {
    'name':name,
    'model_var':model_var,
    'test_accuracy':accuracy_score(predict_DC_tst,target_tst),
    'validate_accuracy':accuracy_score(predict_DC_vld,target_vld)}
top_models = top_models.append(model_data, ignore_index=True)

C:\Users\lyser\AppData\Local\Temp\ipykernel_34316\314042525.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_models = top_models.append(model_data, ignore_index=True)


In [27]:
top_models

,name,model_var,test_accuracy,validate_accuracy
0,Логическая Регрессия,NaN,0.738725,0.713841
1,Дерево Решений,m_depth: 10,0.794712,0.777605
2,Случайный Лес,"(n_est: 21, m_depth: 10)",0.804044,0.807154
3,Dummy классификатор,NaN,0.674961,0.668740


### Вывод

Все наши модели показывают более высокую точность, а значит дают более качественные результаты.

## Общий вывод

Исходный набор данных представляет собой таблицу (3214x5), в которой столбец is_ultra является целевым для задачи бинарной классификации. Данные уже прошли предобработку, мы только сменили тип данных в столбцах calls и messages на int64. Соотношение пользователей тарифов: «Ультра» — 30%, «Смарт» — 70%. Это говорит о том, что у нас в целевом признаке присутствует дисбаланс, то есть наша модель будет предвзята в сторону тарифа «Смарт». Предполагаю, что дисбаланс недостаточно ярковыраженн, чтобы была необходимость с ним бороться.

Мы разбили датафрейм на 3 выборки: обучающую - 60% (df_trn), валидационную - 20% (df_vld) и тестовую - 20% (df_tst).

После исследования 3-х моделей, мы получили 590 интераций. Лучшей оказалась Случайный лес с точностью 0.807 при значениях: максимальной глубины деревьев 10, числа деревьев 25 и ограничения на число объектов в листьях 2.

После проверки наших моделей на тестовых выборках, мы определили, что модели Дерево решений и Случайный лес соответствуют условию точности (доля правильных ответов более 75%).

Для проверки модели на адекватность, мы создали baseline-модель (Dummy классификатор). Все наши модели показывают более высокую точность, а значит дают более качественные результаты, а значит наши модели адекватны.